In [ ]:
import torch 
from torch import nn

# 0 0     3 4
# 0 1     5 6

X = torch.tensor([[0, 0], [0, 1]])
K = torch.tensor([[3, 4], [5, 6]])
# 这个*是对应位置做*得到
y = (X * K)
print(y)

In [ ]:
X = torch.tensor([[0, 1, 2], [3, 4, 5], [6, 7, 8]])
K = torch.tensor([[0, 1], [2, 3]])

# 0 1 2        0 1        19 25
# 3 4 5        2 3    =   37 43
# 6 7 8                   

"""
二维卷积的计算函数，参照上图，注意每个元素的计算跟矩阵计算不是一样的。
"""
def corr2d(X, K): 
    h, w = K.shape
    X, K = X.float(), K.float()
    Y = torch.zeros((X.shape[0] - h + 1, X.shape[1] - w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i: i + h, j: j + w] * K).sum()
    return Y

print(corr2d(X, K))

In [ ]:
"""
检测图像中物体的边缘，即找到像素变化的位置。
构造一张6 × 8的图像(即高和宽分别为6像素和8像素的图像)。它中间4列为黑(0)，其余为白。
构造一个高和宽分别为1和2的卷积核K。
当它与输入做互相关运算时，如果横向相邻元素相同，输出为0;否则输出为非0。
从白到黑的边缘和从黑到白 的边缘分别检测成了1和-1。其余部分的输出全是0。
"""
X = torch.ones(6, 8)
X[:, 2:6] = 0
print(X)
edge_K = torch.tensor([[1, -1]])
Y = corr2d(X, edge_K)
print(Y)

In [ ]:
"""
学习edge_K
学习的前提是我要知道卷积核的形状？
"""
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super(Conv2D, self).__init__()
        self.weight = nn.Parameter(torch.randn(kernel_size))
        self.bias = nn.Parameter(torch.randn(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias

# 构造一个核数组形状是(1, 2)的二维卷积层
conv2d = Conv2D(kernel_size=(1, 2))
print(conv2d)
step = 20
lr = 0.01
for i in range(step):
    Y_hat = conv2d(X)
    l = ((Y_hat - Y) ** 2).sum()
    l.backward()   
    # 梯度下降
    conv2d.weight.data -= lr * conv2d.weight.grad
    conv2d.bias.data -= lr * conv2d.bias.grad
    # 梯度清0
    conv2d.weight.grad.fill_(0)
    conv2d.bias.grad.fill_(0)
    if (i + 1) % 5 == 0:
        print('Step %d, loss %.3f' % (i + 1, l.item()))
        
print("weight: ", conv2d.weight.data)
print("bias: ", conv2d.bias.data)